# Error Analysis


In [1]:
import sys
from pathlib import Path
import glob
import os

sys.path.append(str(Path.cwd().parent))
from src.analysis.analyze_errors import ErrorAnalyzer

project_root = Path.cwd().parent 


### Basic Setup

In [2]:
file_path = project_root / "results/rag_eval_full_single_queries_20260112_200932.json" 

analyzer = ErrorAnalyzer(file_path)

analyzer.check_near_misses()
analyzer.categorize_failures()
analyzer.analyze_multi_turn_decay()

Loaded 747 records for analysis.

RETRIEVAL: Exact Hits vs. Near Misses
----------------------------------------
Total Retrieval Misses: 54
Near Misses (Correct Doc, Wrong Chunk): 33
-> 61.1% of misses were actually close!

RAG BEHAVIOR CATEGORIES
----------------------------------------
Success (Retrieval + Reasoning): 675 (90.4%)
Lucky Guess (Internal Knowledge used): 29 (3.9%)
System Failure (Retrieval missed & Model didn't know): 25 (3.3%)
Context Ignored (Found doc, but failed to answer): 18 (2.4%)


In [4]:
file_path = project_root / "results/rag_eval_full_multi_turn_20260112_211633.json" 

analyzer = ErrorAnalyzer(file_path)

analyzer.check_near_misses()
analyzer.categorize_failures()
analyzer.analyze_multi_turn_decay()

Loaded 228 records for analysis.

RETRIEVAL: Exact Hits vs. Near Misses
----------------------------------------
Total Retrieval Misses: 63
Near Misses (Correct Doc, Wrong Chunk): 40
-> 63.5% of misses were actually close!

RAG BEHAVIOR CATEGORIES
----------------------------------------
Success (Retrieval + Reasoning): 154 (67.5%)
Lucky Guess (Internal Knowledge used): 32 (14.0%)
System Failure (Retrieval missed & Model didn't know): 31 (13.6%)
Context Ignored (Found doc, but failed to answer): 11 (4.8%)

Multi-Turn Performance Decay
----------------------------------------
            rag_score baseline_score hit_rate
turn_number                                  
1                8.13           5.42    84.5%
2                7.03           7.14    70.4%
3                6.94           7.28    62.3%
4                6.47           6.41    70.6%

Context Dependency Check:
Turn 1: RAG beat Baseline in 49/71 cases
Turn 2: RAG beat Baseline in 26/71 cases
Turn 3: RAG beat Baseline in 29/6